# Install Dependencies

In [1]:
!pip install transformers scipy diffusers accelerate opencv-python pandas tqdm typed-argument-parser

In [2]:
!pip show torch

Name: torch
Version: 2.4.0
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3
Location: d:\GitHub\generative-interior-design\.venv\Lib\site-packages
Requires: filelock, fsspec, jinja2, networkx, setuptools, sympy, typing-extensions
Required-by: accelerate


In [3]:
!pip install natten==0.17.1+torch240cu124 -f https://shi-labs.com/natten/wheels/

^C


Looking in links: https://shi-labs.com/natten/wheels/


ERROR: Ignored the following yanked versions: 0.14.1
ERROR: Could not find a version that satisfies the requirement natten==0.17.1+torch240cu124 (from versions: 0.14.2.post4, 0.14.4, 0.14.5, 0.14.6, 0.15.0, 0.15.1, 0.17.0, 0.17.1)
ERROR: No matching distribution found for natten==0.17.1+torch240cu124


In [ ]:
!pip show natten

# Prepare Images Folder

# Create Empty Room Images

## Segmentation Colors py file contents

In [ ]:
COLOR_MAPPING_ = {
    '#FFFFFF': 'background',
    "#787878": "wall",
    "#B47878": "building;edifice",
    "#06E6E6": "sky",
    "#503232": "floor;flooring",
    "#04C803": "tree",
    "#787850": "ceiling",
    "#8C8C8C": "road;route",
    "#CC05FF": "bed",
    "#E6E6E6": "windowpane;window",
    "#04FA07": "grass",
    "#E005FF": "cabinet",
    "#EBFF07": "sidewalk;pavement",
    "#96053D": "person;individual;someone;somebody;mortal;soul",
    "#787846": "earth;ground",
    "#08FF33": "door;double;door",
    "#FF0652": "table",
    "#8FFF8C": "mountain;mount",
    "#CCFF04": "plant;flora;plant;life",
    "#FF3307": "curtain;drape;drapery;mantle;pall",
    "#CC4603": "chair",
    "#0066C8": "car;auto;automobile;machine;motorcar",
    "#3DE6FA": "water",
    "#FF0633": "painting;picture",
    "#0B66FF": "sofa;couch;lounge",
    "#FF0747": "shelf",
    "#FF09E0": "house",
    "#0907E6": "sea",
    "#DCDCDC": "mirror",
    "#FF095C": "rug;carpet;carpeting",
    "#7009FF": "field",
    "#08FFD6": "armchair",
    "#07FFE0": "seat",
    "#FFB806": "fence;fencing",
    "#0AFF47": "desk",
    "#FF290A": "rock;stone",
    "#07FFFF": "wardrobe;closet;press",
    "#E0FF08": "lamp",
    "#6608FF": "bathtub;bathing;tub;bath;tub",
    "#FF3D06": "railing;rail",
    "#FFC207": "cushion",
    "#FF7A08": "base;pedestal;stand",
    "#00FF14": "box",
    "#FF0829": "column;pillar",
    "#FF0599": "signboard;sign",
    "#0633FF": "chest;of;drawers;chest;bureau;dresser",
    "#EB0CFF": "counter",
    "#A09614": "sand",
    "#00A3FF": "sink",
    "#8C8C8C": "skyscraper",
    "#FA0A0F": "fireplace;hearth;open;fireplace",
    "#14FF00": "refrigerator;icebox",
    "#1FFF00": "grandstand;covered;stand",
    "#FF1F00": "path",
    "#FFE000": "stairs;steps",
    "#99FF00": "runway",
    "#0000FF": "case;display;case;showcase;vitrine",
    "#FF4700": "pool;table;billiard;table;snooker;table",
    "#00EBFF": "pillow",
    "#00ADFF": "screen;door;screen",
    "#1F00FF": "stairway;staircase",
    "#0BC8C8": "river",
    "#FF5200": "bridge;span",
    "#00FFF5": "bookcase",
    "#003DFF": "blind;screen",
    "#00FF70": "coffee;table;cocktail;table",
    "#00FF85": "toilet;can;commode;crapper;pot;potty;stool;throne",
    "#FF0000": "flower",
    "#FFA300": "book",
    "#FF6600": "hill",
    "#C2FF00": "bench",
    "#008FFF": "countertop",
    "#33FF00": "stove;kitchen;stove;range;kitchen;range;cooking;stove",
    "#0052FF": "palm;palm;tree",
    "#00FF29": "kitchen;island",
    "#00FFAD": "computer;computing;machine;computing;device;data;processor;electronic;computer;information;processing;system",
    "#0A00FF": "swivel;chair",
    "#ADFF00": "boat",
    "#00FF99": "bar",
    "#FF5C00": "arcade;machine",
    "#FF00FF": "hovel;hut;hutch;shack;shanty",
    "#FF00F5": "bus;autobus;coach;charabanc;double-decker;jitney;motorbus;motorcoach;omnibus;passenger;vehicle",
    "#FF0066": "towel",
    "#FFAD00": "light;light;source",
    "#FF0014": "truck;motortruck",
    "#FFB8B8": "tower",
    "#001FFF": "chandelier;pendant;pendent",
    "#00FF3D": "awning;sunshade;sunblind",
    "#0047FF": "streetlight;street;lamp",
    "#FF00CC": "booth;cubicle;stall;kiosk",
    "#00FFC2": "television;television;receiver;television;set;tv;tv;set;idiot;box;boob;tube;telly;goggle;box",
    "#00FF52": "airplane;aeroplane;plane",
    "#000AFF": "dirt;track",
    "#0070FF": "apparel;wearing;apparel;dress;clothes",
    "#3300FF": "pole",
    "#00C2FF": "land;ground;soil",
    "#007AFF": "bannister;banister;balustrade;balusters;handrail",
    "#00FFA3": "escalator;moving;staircase;moving;stairway",
    "#FF9900": "ottoman;pouf;pouffe;puff;hassock",
    "#00FF0A": "bottle",
    "#FF7000": "buffet;counter;sideboard",
    "#8FFF00": "poster;posting;placard;notice;bill;card",
    "#5200FF": "stage",
    "#A3FF00": "van",
    "#FFEB00": "ship",
    "#08B8AA": "fountain",
    "#8500FF": "conveyer;belt;conveyor;belt;conveyer;conveyor;transporter",
    "#00FF5C": "canopy",
    "#B800FF": "washer;automatic;washer;washing;machine",
    "#FF001F": "plaything;toy",
    "#00B8FF": "swimming;pool;swimming;bath;natatorium",
    "#00D6FF": "stool",
    "#FF0070": "barrel;cask",
    "#5CFF00": "basket;handbasket",
    "#00E0FF": "waterfall;falls",
    "#70E0FF": "tent;collapsible;shelter",
    "#46B8A0": "bag",
    "#A300FF": "minibike;motorbike",
    "#9900FF": "cradle",
    "#47FF00": "oven",
    "#FF00A3": "ball",
    "#FFCC00": "food;solid;food",
    "#FF008F": "step;stair",
    "#00FFEB": "tank;storage;tank",
    "#85FF00": "trade;name;brand;name;brand;marque",
    "#FF00EB": "microwave;microwave;oven",
    "#F500FF": "pot;flowerpot",
    "#FF007A": "animal;animate;being;beast;brute;creature;fauna",
    "#FFF500": "bicycle;bike;wheel;cycle",
    "#0ABED4": "lake",
    "#D6FF00": "dishwasher;dish;washer;dishwashing;machine",
    "#00CCFF": "screen;silver;screen;projection;screen",
    "#1400FF": "blanket;cover",
    "#FFFF00": "sculpture",
    "#0099FF": "hood;exhaust;hood",
    "#0029FF": "sconce",
    "#00FFCC": "vase",
    "#2900FF": "traffic;light;traffic;signal;stoplight",
    "#29FF00": "tray",
    "#AD00FF": "ashcan;trash;can;garbage;can;wastebin;ash;bin;ash-bin;ashbin;dustbin;trash;barrel;trash;bin",
    "#00F5FF": "fan",
    "#4700FF": "pier;wharf;wharfage;dock",
    "#7A00FF": "crt;screen",
    "#00FFB8": "plate",
    "#005CFF": "monitor;monitoring;device",
    "#B8FF00": "bulletin;board;notice;board",
    "#0085FF": "shower",
    "#FFD600": "radiator",
    "#19C2C2": "glass;drinking;glass",
    "#66FF00": "clock",
    "#5C00FF": "flag",
}

def to_rgb(color: str) -> tuple:
    return tuple(int(color[i:i+2], 16) for i in (1, 3, 5))

COLOR_MAPPING_RGB = {to_rgb(k): v for k, v in COLOR_MAPPING_.items()}


def map_colors_rgb(color: tuple) -> str:
    return COLOR_MAPPING_RGB[color]



def ade_palette() -> list[list[int]]:
    """ADE20K palette that maps each class to RGB values."""
    return [[120, 120, 120], [180, 120, 120], [6, 230, 230], [80, 50, 50],
            [4, 200, 3], [120, 120, 80], [140, 140, 140], [204, 5, 255],
            [230, 230, 230], [4, 250, 7], [224, 5, 255], [235, 255, 7],
            [150, 5, 61], [120, 120, 70], [8, 255, 51], [255, 6, 82],
            [143, 255, 140], [204, 255, 4], [255, 51, 7], [204, 70, 3],
            [0, 102, 200], [61, 230, 250], [255, 6, 51], [11, 102, 255],
            [255, 7, 71], [255, 9, 224], [9, 7, 230], [220, 220, 220],
            [255, 9, 92], [112, 9, 255], [8, 255, 214], [7, 255, 224],
            [255, 184, 6], [10, 255, 71], [255, 41, 10], [7, 255, 255],
            [224, 255, 8], [102, 8, 255], [255, 61, 6], [255, 194, 7],
            [255, 122, 8], [0, 255, 20], [255, 8, 41], [255, 5, 153],
            [6, 51, 255], [235, 12, 255], [160, 150, 20], [0, 163, 255],
            [140, 140, 140], [250, 10, 15], [20, 255, 0], [31, 255, 0],
            [255, 31, 0], [255, 224, 0], [153, 255, 0], [0, 0, 255],
            [255, 71, 0], [0, 235, 255], [0, 173, 255], [31, 0, 255],
            [11, 200, 200], [255, 82, 0], [0, 255, 245], [0, 61, 255],
            [0, 255, 112], [0, 255, 133], [255, 0, 0], [255, 163, 0],
            [255, 102, 0], [194, 255, 0], [0, 143, 255], [51, 255, 0],
            [0, 82, 255], [0, 255, 41], [0, 255, 173], [10, 0, 255],
            [173, 255, 0], [0, 255, 153], [255, 92, 0], [255, 0, 255],
            [255, 0, 245], [255, 0, 102], [255, 173, 0], [255, 0, 20],
            [255, 184, 184], [0, 31, 255], [0, 255, 61], [0, 71, 255],
            [255, 0, 204], [0, 255, 194], [0, 255, 82], [0, 10, 255],
            [0, 112, 255], [51, 0, 255], [0, 194, 255], [0, 122, 255],
            [0, 255, 163], [255, 153, 0], [0, 255, 10], [255, 112, 0],
            [143, 255, 0], [82, 0, 255], [163, 255, 0], [255, 235, 0],
            [8, 184, 170], [133, 0, 255], [0, 255, 92], [184, 0, 255],
            [255, 0, 31], [0, 184, 255], [0, 214, 255], [255, 0, 112],
            [92, 255, 0], [0, 224, 255], [112, 224, 255], [70, 184, 160],
            [163, 0, 255], [153, 0, 255], [71, 255, 0], [255, 0, 163],
            [255, 204, 0], [255, 0, 143], [0, 255, 235], [133, 255, 0],
            [255, 0, 235], [245, 0, 255], [255, 0, 122], [255, 245, 0],
            [10, 190, 212], [214, 255, 0], [0, 204, 255], [20, 0, 255],
            [255, 255, 0], [0, 153, 255], [0, 41, 255], [0, 255, 204],
            [41, 0, 255], [41, 255, 0], [173, 0, 255], [0, 245, 255],
            [71, 0, 255], [122, 0, 255], [0, 255, 184], [0, 92, 255],
            [184, 255, 0], [0, 133, 255], [255, 214, 0], [25, 194, 194],
            [102, 255, 0], [92, 0, 255]]

## Libraires Import and Loading Models


In [ ]:
from transformers import OneFormerProcessor, OneFormerForUniversalSegmentation
from PIL import Image, ImageFilter
import numpy as np
import torch
from typing import Union
from diffusers import AutoPipelineForInpainting, DEISMultistepScheduler, StableDiffusionInpaintPipeline
from transformers import pipeline
from transformers import AutoImageProcessor, AutoModelForDepthEstimation
import cv2
from glob import glob
import os
from tqdm import tqdm

processor = OneFormerProcessor.from_pretrained("shi-labs/oneformer_ade20k_dinat_large")
model = OneFormerForUniversalSegmentation.from_pretrained("shi-labs/oneformer_ade20k_dinat_large")
model = model.cuda()

pipe = AutoPipelineForInpainting.from_pretrained('lykon/absolute-reality-1.6525-inpainting', torch_dtype=torch.float16, variant="fp16")
pipe.scheduler = DEISMultistepScheduler.from_config(pipe.scheduler.config)
pipe = pipe.to("cuda")

depth_image_processor = AutoImageProcessor.from_pretrained("LiheYoung/depth-anything-large-hf", torch_dtype=torch.float16)
depth_model = AutoModelForDepthEstimation.from_pretrained("LiheYoung/depth-anything-large-hf", torch_dtype=torch.float16)
depth_model = depth_model.cuda()

## Helper Functions

In [ ]:
def resize_image(input_image, resolution, interpolation=None):
    H, W, C = input_image.shape
    H = float(H)
    W = float(W)
    k = float(resolution) / max(H, W)
    H *= k
    W *= k
    H = int(np.round(H / 64.0)) * 64
    W = int(np.round(W / 64.0)) * 64
    if interpolation is None:
        interpolation = cv2.INTER_LANCZOS4 if k > 1 else cv2.INTER_AREA
    img = cv2.resize(input_image, (W, H), interpolation=interpolation)
    return img

def HWC3(x):
    assert x.dtype == np.uint8
    if x.ndim == 2:
        x = x[:, :, None]
    assert x.ndim == 3
    H, W, C = x.shape
    assert C == 1 or C == 3 or C == 4
    if C == 3:
        return x
    if C == 1:
        return np.concatenate([x, x, x], axis=2)
    if C == 4:
        color = x[:, :, 0:3].astype(np.float32)
        alpha = x[:, :, 3:4].astype(np.float32) / 255.0
        y = color * alpha + 255.0 * (1.0 - alpha)
        y = y.clip(0, 255).astype(np.uint8)
        return y


def get_segmentation_of_room(image: Image) -> tuple[np.ndarray, Image]:
    # Semantic Segmentation
    with torch.inference_mode():
        semantic_inputs = processor(images=image, task_inputs=["semantic"], return_tensors="pt")
        semantic_inputs = {key: value.to("cuda") for key, value in semantic_inputs.items()}
        semantic_outputs = model(**semantic_inputs)
        # pass through image_processor for postprocessing
        predicted_semantic_map = \
        processor.post_process_semantic_segmentation(semantic_outputs, target_sizes=[image.size[::-1]])[0]

    predicted_semantic_map = predicted_semantic_map.cpu()
    color_seg = np.zeros((predicted_semantic_map.shape[0], predicted_semantic_map.shape[1], 3), dtype=np.uint8)

    palette = np.array(ade_palette())
    for label, color in enumerate(palette):
        color_seg[predicted_semantic_map == label, :] = color

    color_seg = color_seg.astype(np.uint8)
    seg_image = Image.fromarray(color_seg).convert('RGB')
    return color_seg, seg_image


def filter_items(
    colors_list: Union[list, np.ndarray],
    items_list: Union[list, np.ndarray],
    items_to_remove: Union[list, np.ndarray]
):
    filtered_colors = []
    filtered_items = []
    for color, item in zip(colors_list, items_list):
        if item not in items_to_remove:
            filtered_colors.append(color)
            filtered_items.append(item)
    return filtered_colors, filtered_items


def get_inpating_mask(segmentation_mask: np.ndarray) -> Image:
    unique_colors = np.unique(segmentation_mask.reshape(-1, segmentation_mask.shape[2]), axis=0)
    unique_colors = [tuple(color) for color in unique_colors]
    segment_items = [map_colors_rgb(i) for i in unique_colors]

    control_items = ["windowpane;window", "wall", "floor;flooring","ceiling",  "sconce", "door;double;door", "light;light;source",
                     "painting;picture", "stairs;steps","escalator;moving;staircase;moving;stairway"]
    chosen_colors, segment_items = filter_items(
                colors_list=unique_colors,
                items_list=segment_items,
                items_to_remove=control_items
            )

    mask = np.zeros_like(segmentation_mask)
    for color in chosen_colors:
        color_matches = (segmentation_mask == color).all(axis=2)
        mask[color_matches] = 1

    mask_image = Image.fromarray((mask * 255).astype(np.uint8)).convert("RGB")
    # enlarge mask region so that it also will erase the neighborhood of masked stuff
    mask_image = mask_image.filter(ImageFilter.MaxFilter(25))
    return mask_image

def cleanup_room(image: Image, mask: Image) -> Image:
    inpaint_prompt = "Empty room, with only empty walls, floor, ceiling, doors, windows"
    negative_prompt = "furnitures, sofa, cough, table, plants, rug, home equipment, music equipment, shelves, books, light, lamps, window, radiator"
    image_source_for_inpaint = image.resize((512, 512))
    image_mask_for_inpaint = mask.resize((512, 512))
    generator = [torch.Generator(device="cuda").manual_seed(20)]

    image_inpainting_auto = \
    pipe(prompt=inpaint_prompt, negative_prompt=negative_prompt, generator=generator, strentgh=0.8,
         image=image_source_for_inpaint, mask_image=image_mask_for_inpaint, guidance_scale=10.0,
         num_inference_steps=10).images[0]
    image_inpainting_auto = image_inpainting_auto.resize((image.size[0], image.size[1]))
    return image_inpainting_auto


def get_depth_image(image: Image) -> Image:
    image_to_depth = depth_image_processor(images=image, return_tensors="pt").to("cuda")
    with torch.no_grad():
        depth_map = depth_model(**image_to_depth).predicted_depth

    width, height = image.size
    depth_map = torch.nn.functional.interpolate(
        depth_map.unsqueeze(1).float(),
        size=(height, width),
        mode="bicubic",
        align_corners=False,
    )
    depth_min = torch.amin(depth_map, dim=[1, 2, 3], keepdim=True)
    depth_max = torch.amax(depth_map, dim=[1, 2, 3], keepdim=True)
    depth_map = (depth_map - depth_min) / (depth_max - depth_min)
    image = torch.cat([depth_map] * 3, dim=1)

    image = image.permute(0, 2, 3, 1).cpu().numpy()[0]
    image = Image.fromarray((image * 255.0).clip(0, 255).astype(np.uint8))
    return image

## Main

In [ ]:
if __name__ == "__main__":
    for room in os.listdir(r"C:\Users\hassa\Downloads\archive\House_Room_Dataset"):
      save_dir = os.path.join(r"D:\Musketeers\Room Furniture Adjuster\Dataset", room)
      image_paths = os.path.join(r"C:\Users\hassa\Downloads\archive\House_Room_Dataset", room)
      all_images = glob(f"{image_paths}/*")
      os.makedirs(save_dir, exist_ok=True)

      for elem in tqdm(all_images):
          image_id = os.path.basename(elem).split(".")[0]
          image = Image.open(elem)
          color_map, segmentation_map = get_segmentation_of_room(image)
          inpaiting_mask = get_inpating_mask(color_map)
          clean_room = cleanup_room(image, inpaiting_mask)
          color_map_clean, segmentation_map_clean_room = get_segmentation_of_room(clean_room)
          depth_clean_room = get_depth_image(clean_room)

          clean_room.save(os.path.join(save_dir, image_id+"_empty.png"))
          # segmentation_map_clean_room.save(os.path.join(save_dir, image_id+"segmentation.png"))
          # depth_clean_room.save(os.path.join(save_dir, image_id+"depth.png"))